In [24]:
import requests
import os
api_key = "08cf729e7e5641f8bc00fa590017cc5ebc1e9397ac6a8491a671b4bf03f55c5b"
path_file = "protected_archive/invoice-42369643.html"
url = "https://www.virustotal.com/api/v3/files"

if not os.path.exists(path_file):
    print(f"Ошибка: Файл {path_file} не найден!")
    exit()
else:
    print("Файл существует")
try:
    with open(path_file, "rb") as file:
        # Формируем заголовки и файл
        headers = {
            "accept": "application/json",
            "x-apikey": api_key
        }
        files = {"file": (path_file, file, "text/html")}

        response = requests.post(
            url,
            files=files,
            headers=headers,
            timeout=10  # Таймаут на случай проблем с сетью
        )

    print("Status Code:", response.status_code)
    print("Response:", response.text)

except FileNotFoundError:
    print(f"Ошибка: Файл {path_file} не найден!")
except requests.exceptions.RequestException as e:
    print(f"Ошибка запроса: {e}")
except Exception as e:
    print(f"Неизвестная ошибка: {e}")

Файл существует
Status Code: 200
Response: {"data": {"type": "analysis", "id": "NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjgzNzgxMg==", "links": {"self": "https://www.virustotal.com/api/v3/analyses/NjEyYjU3ZTJhZmU3MDY1ZWJlOTIxMzM3MTcwZGY3ZDQ6MTc0MjgzNzgxMg=="}}}


In [25]:
import time
data = response.json()
list_data = data["data"]
id_file = list_data["id"]
url_get = f"https://www.virustotal.com/api/v3/analyses/{id_file}"
def get_analysis_results():
    max_count = 10
    count = 0
    while count <  max_count:
        get_file = requests.get(url_get, headers=headers)
        data = get_file.json()

        status = data["data"]["attributes"]["status"]

        if status == "completed":
            print("Анализ завершён!")
            return data
        
        print(f"Статус: {status} (попытка {count + 1}/{max_count})")
        time.sleep(30)  # Ждем 30 секунд перед следующей проверкой, так как он достаточно долго проверяет, иногда не попадаем вовремя
        count += 1

    raise Exception("Анализ не завершился в течение отведенного времени")

try:
    results = get_analysis_results()
    
    # Выводим основные результаты
    attributes = results["data"]["attributes"]
    stats = attributes["stats"]

    # Сколько обнаружило и сколько не обнаружило
    print("=== STATS ===")
    print(f"Вредоносных: {stats['malicious']}")
    print(f"Не обнаружено: {stats['undetected']}")
    print("\n=== ОБНАРУЖИЛИ ВРЕДОНОС ===")
    result_list = attributes["results"]
    detected = [engine for engine, result in result_list.items() if result.get("category") == "malicious"]
    print(", ".join(detected))

    #В задании указано посмотреть на данные 
    specific_avs = ["Fortinet", "McAfee", "Yandex", "Sophos"]
    print("\n=== АНАЛИЗ УКАЗАННЫХ В ЗАДАНИИ АНТИВИРУСОВ ===")
    for av in specific_avs:
        if av in result_list:
            status = "Обнаружен вредонос" if result_list[av]["category"] == "malicious" else "Не обнаружено"
            result = result_list[av].get("result", "N/A")
            print(f"{av}: {status} ({result})")
        else:
            print(f"{av}: Такого антивируса в списке нет")
except Exception as e:
    print(f"Ошибка: {e}")

#print(id_file)
#print(url)
#get_file = requests.get(url_get, headers=headers)

#print(get_file.text)

Анализ завершён!
=== STATS ===
Вредоносных: 26
Не обнаружено: 36

=== ОБНАРУЖИЛИ ВРЕДОНОС ===
Lionic, MicroWorld-eScan, ClamAV, CTX, Sangfor, Symantec, ESET-NOD32, Avast, Kaspersky, BitDefender, Emsisoft, DrWeb, VIPRE, FireEye, Google, Fortinet, Kingsoft, Arcabit, ViRobot, Microsoft, Varist, ALYac, Ikarus, GData, AVG, alibabacloud

=== АНАЛИЗ УКАЗАННЫХ В ЗАДАНИИ АНТИВИРУСОВ ===
Fortinet: Обнаружен вредонос (VBA/Agent.BCF0!tr)
McAfee: Не обнаружено (None)
Yandex: Не обнаружено (None)
Sophos: Не обнаружено (None)


Vulners

In [ ]:
import vulners
import json
import requests
import time
api_key_vulners = "493BI85EBNVLURYAV7LEOMS8M16CYU5BAUA1TWW7LJMY6WXVL5U76NSDIVXZFFTB"
vulners_api = vulners.Vulners(api_key=api_key_vulners)
list_software = "vulners_temp/list.json"

def load_software_data():
    try:
        with open(list_software, 'r', encoding='utf-8') as f:
            data = json.load(f)
            #print(data)
            if not isinstance(data, list):
                raise ValueError("JSON должен содержать массив объектов")
            return data
    except Exception as e:
        print(f"🚨 Ошибка загрузки JSON: {e}")
        return None

def process_vulnerability_results(data):
    results = []
    
    for vuln_data in data:
        if not vuln_data:  # Skip empty results
            continue
            
        for item in vuln_data:
            exploit_flag = is_exploit(item)

            result = {
                'title': item.get('title', 'No title'),
                'type': item.get('type', 'Unknown'),
                'cves': item.get('cvelist', []),
                'cvss': item.get('cvss', {}).get('score', 0),
                'published': item.get('published', ''),
                'description': item.get('description', 'No description'),
                'source': item.get('href', 'No source'),
                'is_exploit': exploit_flag
            }
            results.append(result)
    
    # Sort by CVSS score (highest first)
    return sorted(results, key=lambda x: x['cvss'], reverse=True)

def print_vulnerability_report(processed_results):
    print("\nVulnerability Report:")
    print("="*50)
    
    for i, result in enumerate(processed_results, 1):
        print(f"\n{i}. {result['title']}")
        print(f"Type: {result['type']}")
        print(f"CVSS Score: {result['cvss']}")
        print(f"Published: {result['published']}")
        print(f"CVEs: {', '.join(result['cves']) if result['cves'] else 'None'}")
        
        # Print truncated description
        desc = result['description'][:200] + "..." if len(result['description']) > 200 else result['description']
        print(f"Description: {desc}")
        
        print(f"Source: {result['source']}")

def is_exploit(item):
    exploit_indicators = [
        'exploit' in item.get('type', '').lower(),
        'exploit-db.com' in item.get('href', ''),
        '0day.today' in item.get('href', ''),
        'packetstormsecurity.com' in item.get('href', ''),
        'metasploit' in item.get('description', '').lower(),
        any(x in item.get('title', '').lower() for x in ['exploit', 'poc', 'proof of concept']),
        'coresecurity.com' in item.get('href', '') and ('advisory' in item.get('href', '') or 'advisories' in item.get('href', '')),
        'thehackernews.com' in item.get('href', '') and 'exploit' in item.get('description', '').lower()
    ]
    return any(exploit_indicators)

data = load_software_data()
result = []

for item in data:
    test = vulners_api.find_all(f'"{item["Program"]} {item["Version"]}" AND cvss.score:[5 TO 10]', limit=5)
    result.append(test)
#test_results = vulners_api.search('"LibreOffice 6.0.7" AND cvss.score:[5 TO 10]', limit=5)
#print(test_results)
print("\n=== НАЙДЕННЫЕ УЯЗВИМОСТИ ===")
processed_results = process_vulnerability_results(result)
print_vulnerability_report(processed_results)
#--------------------
total_cves = sum(len(res['cves']) for res in processed_results)
exploit_cves = [cve for res in processed_results for cve in res.get('cves', []) if res.get('is_exploit', False)]
exploit_details = [f"{res.get('title', 'No title')} ({', '.join(res.get('cves', []))})" 
                  for res in processed_results if res.get('is_exploit', False)]
print("\n=== SUMMARY ===")
print(f"Total vulnerabilities: {len(processed_results)}")
print(f"Total CVEs found: {total_cves}")
print(f"Vulnerabilities with known exploits: {len(exploit_cves)}")
if exploit_cves:
    print("CVEs with exploits:", ', '.join(exploit_cves))
if exploit_details:
    print("\nExploits available for:")
    for detail in exploit_details:
        print(f"- {detail}")

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\vulners\vulners.py:838: DeprecationWarning: Vulners is deprecated and will be removed in future release. Use VulnersApi instead.
  warnings.warn(



=== НАЙДЕННЫЕ УЯЗВИМОСТИ ===

Vulnerability Report:

1. RHSA-2019:0367 Red Hat Security Advisory: Red Hat JBoss Core Services Apache HTTP Server 2.4.29 security update
Type: osv
CVSS Score: 9.8
Published: 2024-09-13T16:54:56
CVEs: CVE-2017-15710, CVE-2017-15715, CVE-2018-0739, CVE-2018-1000168, CVE-2018-11759, CVE-2018-11763, CVE-2018-1283, CVE-2018-1301, CVE-2018-1302, CVE-2018-1303, CVE-2018-1312, CVE-2018-1333
Description: Bulletin has no description
Source: https://osv.dev/vulnerability/RHSA-2019:0367

2. RHSA-2018:2185 Red Hat Security Advisory: Red Hat JBoss Core Services Apache HTTP Server 2.4.29  RHEL 7 security update
Type: osv
CVSS Score: 9.8
Published: 2024-09-13T14:43:20
CVEs: CVE-2016-2182, CVE-2016-4975, CVE-2016-6302, CVE-2016-6306, CVE-2016-7055, CVE-2017-3731, CVE-2017-3732, CVE-2017-3736, CVE-2017-3737, CVE-2017-3738
Description: Bulletin has no description
Source: https://osv.dev/vulnerability/RHSA-2018:2185

3. RHSA-2018:2186 Red Hat Security Advisory: Red Hat JBos